In [ ]:
# default_exp core

In [ ]:
# Testing the skipgram
import functools
from nltk.util import skipgrams
from sklearn.feature_extraction.text import CountVectorizer
test_txt = ['this is a text block start.', ' This is the end.']
text = [line.strip() for line in test_txt]
print(text)
skipper = functools.partial(skipgrams, n=3, k=2)
list(skipper(text[0]))
vectorizer = CountVectorizer(analyzer=skipper, lowercase=False, min_df=2, strip_accents=False)
vectorizer.fit(text)
vectorizer.vocabulary_

['this is a text block start.', 'This is the end.']


{('t', 'h', ' '): 26,
 ('h', 'i', 's'): 9,
 ('h', 'i', ' '): 7,
 ('h', 'i', 'i'): 8,
 ('h', 's', ' '): 10,
 ('h', 's', 'i'): 11,
 ('h', ' ', 'i'): 6,
 ('i', 's', ' '): 15,
 ('i', 's', 'i'): 16,
 ('i', 's', 's'): 17,
 ('i', ' ', 'i'): 12,
 ('i', ' ', 's'): 13,
 ('i', 'i', 's'): 14,
 ('s', ' ', 'i'): 19,
 ('s', ' ', 's'): 20,
 ('s', ' ', ' '): 18,
 ('s', 'i', 's'): 23,
 ('s', 'i', ' '): 22,
 ('s', 's', ' '): 24,
 (' ', 'i', 's'): 2,
 (' ', 'i', ' '): 1,
 (' ', 's', ' '): 3,
 ('s', ' ', 't'): 21,
 (' ', ' ', 't'): 0,
 (' ', 't', 'e'): 5,
 ('t', 'e', ' '): 25,
 (' ', 's', 't'): 4}

# SGNN

> Implementation of Self-Governing Neural Networks for speech act classification

Implementation of the [SGNN paper](https://www.aclweb.org/anthology/D19-1402.pdf) for speech act classification.
This repository is inspired by Guillaume Chevalier's [implementation](https://github.com/guillaume-chevalier/SGNN-Self-Governing-Neural-Networks-Projection-Layer), as well as his [discussion](https://github.com/guillaume-chevalier/SGNN-Self-Governing-Neural-Networks-Projection-Layer/issues/1) with [Sava Kalbachou](https://github.com/thinline).
This version implements some things differently from Guillaume's code, and extends beyond the projection layer all the way to a fully trainable network.

The network is trained to classify the [SwDA corpus](https://web.stanford.edu/~jurafsky/ws97/) utterances according to their speech act. The corpus was was pre-processed using Cristopher Pott's [project](https://github.com/cgpotts/swda/) related to it; the pre-processed data is included in [`data/swda-acttags-and-text.csv`](data/swda-acttags-and-text.csv) for repeateability.

In [ ]:
#hide
from nbdev.showdoc import *

We first import the data from file

In [ ]:
import pandas as pd
data_filepath = '/home/andres/repositories/SGNN/data/swda-acttags-and-text.csv'

data = pd.read_csv(data_filepath)

In [ ]:
# See the data
data.describe

<bound method NDFrame.describe of        DamslActTag                                               Text
0               ad               So, uh, describe your family budget.
1               sd  Well, I've, uh, for a lot of years I, I've pre...
2               sd  and, uh, just recently, uh, we, we set up a bu...
3               sd              and, and we're trying to stick to it.
4               sd                        We just bought a new house.
...            ...                                                ...
221611           %                                           so, yeah
221612          sd                                               I --
221613           %                                         I don't, -
221614           +  -- pretty much plan on, I mean, so far I'm pla...
221615          ba                                   That's good, uh,

[221616 rows x 2 columns]>

Divide data in test and train sets

In [ ]:
#export
import functools
from nltk.util import skipgrams
import numpy as np
import pandas as pd
import random as rand
import scipy.sparse as sp
from sklearn.base import BaseEstimator
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

from nearpy.hashes import RandomBinaryProjections

In [ ]:
#export
def import_data(filepath):
    data = pd.read_csv(data_filepath)
    data = data.dropna(axis=0)  # Drop rows with NA values
    
    return data.Text, data.DamslActTag
    
    
def preprocess_data(X, y):
    # Convert labels to categories
    le = LabelEncoder()
    y = le.fit_transform(y)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    
    # Convert categories to one-hot-encodings, as apparently needed by keras
    train_labels = tf.keras.utils.to_categorical(y_train)
    test_labels = tf.keras.utils.to_categorical(y_test)
    
    return X_train, X_test, train_labels, test_labels

In [ ]:
X, y = import_data(data_filepath)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

See the data division

In [ ]:
# print(X_train.head(),"\n", y_train.head())

Convert the data labels to one-hot vectors

## Let's build SGNN the pipeline for processing the data

In [ ]:
#export
class MyRBP(BaseEstimator, RandomBinaryProjections):
    rand_seed = None  # Declare it as class variable
    def __init__(self, hash_name='hasher', projection_count=1, rand_seed=None):
        RandomBinaryProjections.__init__(self, hash_name, projection_count, rand_seed=rand_seed)
        
    def fit(self, X, y):
        self.rand = np.random.RandomState(self.rand_seed)  # rand seed after param setting
        self.reset(X.shape[1])
        
    def transform(self, X):
        return self.hash_vector(X)

    def fit_transform(self, X, y):
        self.fit(X, y)
        return self.transform(X)
    
    def hash_vector(self, v, querying=False):
        """
        Hashes the vector and returns the binary bucket key as string.
        """
        if sp.issparse(v):
            # If vector is sparse, make sure we have the CSR representation
            # of the projection matrix
            if self.normals_csr is None:
                self.normals_csr = sp.csr_matrix(self.normals)
            # Make sure that we are using CSR format for multiplication
            if not sp.isspmatrix_csr(v):
                v = sp.csr_matrix(v)
            # Project vector onto all hyperplane normals
            # projection = self.normals_csr.dot(v)
            projection = v.dot(sp.csr_matrix.transpose(self.normals_csr))
        else:
            # Project vector onto all hyperplane normals
            projection = np.dot(v, np.matrix.transpose(self.normals))
        # Return binary key
        return projection > 0

In [ ]:
#export
def build_input_layer(T=80, d=14, char_ngram_range=(1, 4)):
    # T=80 projections for each of dimension d=14: 80 * 14 = 1120-dimensionnal word projections

    skipper = functools.partial(skipgrams, n=3, k=2)
    char_term_frequency_params = {
        'char_term_frequency__analyzer': skipper,
        'char_term_frequency__lowercase': True,
        #  'char_term_frequency__ngram_range': char_ngram_range,
        'char_term_frequency__strip_accents': None,
        'char_term_frequency__min_df': 2,
        'char_term_frequency__max_df': 0.99,
        'char_term_frequency__max_features': int(1e7),
    }

    rand_seeds = [rand.randint(0, T*100) for i in range(T)] # Need a different seed for each hasher

    hashing_feature_union_params = {
        **{'union__random_binary_projection_hasher_{}__projection_count'.format(t): d
           for t in range(T)
        },
        **{'union__random_binary_projection_hasher_{}__hash_name'.format(t): 'hasher' + str(t)
           for t in range(T)
        },
        **{'union__random_binary_projection_hasher_{}__rand_seed'.format(t): rand_seeds[t]  # only AFTER hashing.
           for t in range(T)
        }
    }

    preprocessor = Pipeline([
        ("char_term_frequency", CountVectorizer()),
        ('union', FeatureUnion([
            ('random_binary_projection_hasher_{}'.format(t), MyRBP())
            for t in range(T)
        ]))
    ])
    
    params = dict()
    params.update(char_term_frequency_params)
    params.update(hashing_feature_union_params)
    preprocessor.set_params(**params)
    return preprocessor

In [ ]:
preprocessor = build_input_layer()
preprocessor.fit_transform(["Esta es na pruebs","fadsf", "Oh no fasd", "fasdfaaaaaaaa" ])

<4x1120 sparse matrix of type '<class 'numpy.bool_'>'
	with 2290 stored elements in Compressed Sparse Row format>

In [ ]:
#export
def build_keras_model(train_labels):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(256, activation=tf.nn.sigmoid, input_shape=(1120,)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Dense(256, activation=tf.nn.sigmoid))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Dense(train_labels.shape[1], activation=tf.nn.softmax))

    # Cosine annealing decay
    lr_schedule = tf.keras.experimental.CosineDecay(0.025, decay_steps=1000000)
    # SGD optimizer with Nesterov momentum
    opt = tf.keras.optimizers.SGD(nesterov=True, learning_rate=lr_schedule)
    # opt = tf.keras.optimizers.SGD(nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    return model

/home/andres/src/miniconda3/envs/speechActs/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/andres/src/miniconda3/envs/speechActs/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/andres/src/miniconda3/envs/speechActs/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/andres/src/mi

In [ ]:
#export
def main():
    X, y = import_data(data_filepath)
    X_train, X_test, train_labels, test_labels = preprocess_data(X, y)
    
    input_layer = build_input_layer()
    train_features = input_layer.fit_transform(X_train)
    test_features = input_layer.transform(X_test)
    
    keras_model = build_keras_model(train_labels)
    keras_model.fit(train_features, train_labels, epochs=EPOCHS, batch_size=BATCH_SIZE)
    keras_model.evaluate(test_features, test_labels)

In [ ]:
#export
EPOCHS = 200
BATCH_SIZE = 100
data_filepath = "/home/andres/repositories/SGNN/data/swda-acttags-and-text.csv"
data_filepath = "/home/andres/repositories/SGNN/data/test_swda-acttags-and-text.csv"

main()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/200
177288/177288 [==============================] - 7s 40us/sample - loss: 1.9890 - acc: 0.4613
Epoch 2/200
177288/177288 [==============================] - 7s 40us/sample - loss: 1.6831 - acc: 0.5255
Epoch 3/200
177288/177288 [==============================] - 7s 41us/sample - loss: 1.5889 - acc: 0.5466
Epoch 4/200
177288/177288 [==============================] - 7s 39us/sample - loss: 1.5186 - acc: 0.5582
Epoch 5/200
177288/177288 [==============================] - 7s 40us/sample - loss: 1.4695 - acc: 0.5678
Epoch 6/200
177288/177288 [==============================] - 7s 42us/sample - loss: 1.4299 - acc: 0.5755
Epoch 7/200
177288/177288 [==============================] - 7s 41us/sample - loss: 1.3937 - acc: 0.5856
Epoch 8/200
177288/177288 [==============================] - 7s 42us/sample - loss: 1.3608 - acc: 0.5936
Epoch 9/200
177288/177288 [===============

177288/177288 [==============================] - 7s 37us/sample - loss: 0.9865 - acc: 0.6826
Epoch 151/200
177288/177288 [==============================] - 7s 37us/sample - loss: 0.9853 - acc: 0.6835
Epoch 152/200
177288/177288 [==============================] - 7s 37us/sample - loss: 0.9833 - acc: 0.6829
Epoch 153/200
177288/177288 [==============================] - 7s 37us/sample - loss: 0.9847 - acc: 0.6843
Epoch 154/200
177288/177288 [==============================] - 7s 37us/sample - loss: 0.9827 - acc: 0.6849
Epoch 155/200
177288/177288 [==============================] - 7s 37us/sample - loss: 0.9813 - acc: 0.6847
Epoch 156/200
177288/177288 [==============================] - 7s 37us/sample - loss: 0.9818 - acc: 0.6837
Epoch 157/200
177288/177288 [==============================] - 7s 37us/sample - loss: 0.9810 - acc: 0.6849
Epoch 158/200
177288/177288 [==============================] - 7s 37us/sample - loss: 0.9791 - acc: 0.6850
Epoch 159/200
177288/177288 [======================